In [ ]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="2"

In [ ]:
import torch
from fastai2.basics import *
from fastai2.vision.all import *
from fastai2.data.transforms import *


In [ ]:
import fastai2

In [ ]:
%matplotlib inline

In [ ]:
import pandas as pd
pd.set_option('display.max_rows', 50)
pd.set_option('display.max_columns', 170)
pd.set_option('display.max_colwidth', -1)

In [ ]:
import numpy as np
from matplotlib import pyplot as plt
import openslide
from numpy import asarray

In [ ]:
class TMA:
    diameter_spot = 1200
    def __init__(self, tma_path, file, img_path_tif):
        self.meta_fn = file
        self.meta_path = tma_path
        self.img_path_tif = img_path_tif
        self.name = file.strip('.txt')[len('TMA results - '):]
        self.spots = self.load_spots_meta()
        self.img_tif = self.load_img_tif()
        
    def load_tma_meta(self):
        df1 = pd.read_csv(os.path.join(self.meta_path, self.meta_fn), names=['fn', 'missing','X', 'Y', 'ID'], sep='\t', skiprows=[0])
        return df1
    
    def load_img_tif(self):
        a = None
        try:
            a = openslide.OpenSlide(os.path.join(self.img_path_tif, self.name + '.tif'))
        except: 
            print(f"BAD {self.name}.tif")
        return a
    
    def load_spots_meta(self):
        spots = []
        for index, rows in self.load_tma_meta().iterrows():
            spots.append(Spot(self.diameter_spot, 
                              rows.fn, rows.missing, (rows.X, rows.Y), rows.ID,
                        self.meta_path, self))
        return spots


In [ ]:
class Spot: 
    def __init__(self, diameter, name, missing, center, ID, img_path_jpg, tma):
        self.path_jpg = img_path_jpg
        self.diameter = diameter #in micrometers
        self.name = name
        self.center = center #in micrometers
        self.missing = missing
        self.ID = ID
        self.TMA = tma
        self.tiles = []
        
        
        
    def load_img_jpg(self):
        s = openslide.ImageSlide(os.path.join(self.path_jpg, self.name + '.jpg'))
        #regions to be read with method s.read_region()
        return s
    
    #get spot center coordinates in pixels
    def get_center_pixel(self):
        tma = self.TMA.img_tif
        c_x, c_y = self.center 
        mpp_x, mpp_y = float(tma.properties['openslide.mpp-x']), float(tma.properties['openslide.mpp-y']) #mpp  - 0.25  micras per pixel
        pc_x, pc_y = int(c_x / float(mpp_x)),int(c_y / float(mpp_y))
        return pc_x, pc_y
    
    #return radious in pixels both at full magnification (level = 0) and at a given zoom-out level
    def get_radious_pixel(self, level):
        tma = self.TMA.img_tif
        mpp_x, mpp_y = float(tma.properties['openslide.mpp-x']), float(tma.properties['openslide.mpp-y'])
        #mpp = (mpp_x**2 + mpp_y**2)**0.5
        pspot_radio = int(self.diameter/ mpp_x / 2 )
        pzoom_radio = int( pspot_radio/ 2**level ) #with each increasing level the image is reduced half size 
        return pspot_radio, pzoom_radio
    
    #return spot circunference's north and sud y-coordinates (absolute positions in level 0) 
    #given a relative position from 0 (left) to 1 (right) in the spot diameter in axis x
    def get_circle_coordinates_pixel(self, rel_pos_diameter_x = .5): #return by default the y-coordinates for x fixed at center
        pc_x, pc_y = self.get_center_pixel()
        pspot_radio, _ = self.get_radious_pixel(0)
        x = int(pc_x - pspot_radio + rel_pos_diameter_x*(pspot_radio*2))
        if (( x + pspot_radio < pc_x) or ( x - pspot_radio > pc_x)): #x should be in circle diameter (pc_x - pspot_radio, pc_x + pspot_radio)
            return x, np.nan, np.nan
        root = (pspot_radio**2 - (x - pc_x)**2)**0.5
        y_north = int( -root + pc_y)
        y_sud = int( root + pc_y)
        return x, y_north, y_sud
    
    #check (true or false) if there is scarce tissue in an image
    def check_scarce_tissue(self, tile_image): 
        s = asarray(tile_image)
        return s.mean() >= 230 #completely white = 255
   
    #return an image tile from the spot where x,y is the left-sup corner of the tile
    #for reference for current scanned tifs properties, level 0 is fully magnified image and for level 4 encloses one spot of diameter 1.2 micrometers
    def get_tile(self, x,y , size=(300,300) , level = 4, path = None, tif_name = None):
        tma = None
        if (tif_name == None):
            tma = self.TMA.img_tif
        else:
            try:
                tma = openslide.OpenSlide(os.path.join(path, tif_name + '.tif'))
            except: 
                print(tif_name + '.tif')
        
            
        image = tma.read_region((x,y),level,size)
        if Spot.check_scarce_tissue(Spot, image):
            return np.nan
        else:
            return image

   
        
    
    #check (true or false) that tile (defined by left sup coordinate and size in a zoom level) is enclosed in a box defined by the 4 spot poles 
    def check_tile_in_spot(self, x,y,size=(300,300) , level = 4):
        spot_poles = (self.get_circle_coordinates_pixel(0)[0], #left
                  self.get_circle_coordinates_pixel(1)[0],  #right
                  self.get_circle_coordinates_pixel(0.5)[1], #north
                  self.get_circle_coordinates_pixel(0.5)[2]) #sud
        if (x < spot_poles[0]) or (y < spot_poles[2] ) or (x + size[0]*(2**level) > spot_poles[1] ) or (y + size[1]*(2**level) > spot_poles[3]):
            return False, spot_poles
        else:
            return True, spot_poles
        
        
    #given a tile size in pixels and zoom level then sample all possible tiles from spot 
    #with a given overlap (0 to <1)
    #return all tiles (defined by sup-lef corner coordinates, size and level) 
    #enclosed in box-spot with enough tissue 
    def sample_tiles(self, size = (300,300), level = 4, overlap = .2):
        self.tiles = []
        pspot_radio, _ = self.get_radious_pixel(level)
        spot_tile_ratio = pspot_radio * 2 / size[0] 
        spot_tile_ratio_by_zoom = spot_tile_ratio / 2**level 
        stride_norm = 1 - overlap
        #print(np.arange(0,1,stride_norm/spot_tile_ratio_by_zoom))
        for i_x in np.arange(0,1,stride_norm/spot_tile_ratio_by_zoom): 
            x, y_north, y_sud = self.get_circle_coordinates_pixel(i_x)
            if (y_sud-y_north == 0): #intersection of ecuador with circunference
                if self.check_tile_in_spot(x, int(y_north), size, level)[0]:
                    self.tiles.append([x, int(y_north), size, level])
                if self.check_tile_in_spot(x, int(y_north), size, level)[0]:
                    self.tiles.append([x, int(y_north) - pspot_radio, size, level])
            else:
                for j in np.arange(y_north, y_sud, (y_sud-y_north)*stride_norm/spot_tile_ratio_by_zoom ): 
                    if self.check_tile_in_spot(x, int(j), size, level)[0]:
                        self.tiles.append([x, int(j), size, level])

        return self.tiles
    
    

    #given a zoom level it returns the box enclosing only this spot
    def get_enclosing_box(self, level):
        pspot_radio, pzoom_radio = self.get_radious_pixel(level)
        print(pzoom_radio)
        pc_x, pc_y = spot.get_center_pixel()
        tile = spot.get_tile(pc_x - pspot_radio, pc_y - pspot_radio,  (pzoom_radio * 2, pzoom_radio * 2), level )
        return tile
    

    
        

In [ ]:
path = 'SPOTS/TMA06-03.qptma.data/'
df = pd.read_csv('DTS2020 ALENDA.xlsx - DTS2020_ALENDA.csv', )
df_labels = df[['COD_DTS','Epicolon1+2IHQ-IMS-maria paper lynlike_IMS','LynchIMS','ihq_mlh1','dukes_r',
 'TNMagrup','BaseEP1y2actualizada2016-OSCAR-def_n', 'grado_di','infirec','moc',
'edat', 'sexe','ccr_sin','aden_sin','r_beth_4',
'RECIMORT','KRAS','localizacion','fechreci','ILEact','estfseagrup']]

In [ ]:
import os
tmas = []
for root, dirs, files in os.walk("SPOTS"):
    for file in files:
        if file.endswith(".txt"):
            tma = TMA(root, file, 'DTS2019')
            tmas.append(tma)
            

            

In [ ]:
#generate Dataset 
ds = pd.DataFrame(columns=['tile', 'path','tif_fn', 'patient_ID','label'])

In [ ]:
#size = (300,300), level = 3, overlap = .2

label = 'edat'
for tma in tmas:
    for spot in tma.spots:
        for tile in spot.sample_tiles(size = (300,300), level = 1, overlap = .2):
            s = df_labels.loc[df_labels['COD_DTS'] == str(spot.ID)]
            if (s.shape[0] != 0):
                if label == 'edat':
                    l = float(str(s.iloc[0][label]).replace(',','.'))
                else:
                    l = s.iloc[0][label]
                
                if not np.isnan(l):
                    ds = ds.append({'tile': tile, 'path': tma.img_path_tif, 'tif_fn': tma.name, 'patient_ID': spot.ID,'label': l }, ignore_index=True)          




In [ ]:
d = ds.merge(df_labels, how='left', left_on='patient_ID',right_on='COD_DTS', )
d = d.loc[d['Epicolon1+2IHQ-IMS-maria paper lynlike_IMS'].isna() == False]
d.label = d['Epicolon1+2IHQ-IMS-maria paper lynlike_IMS'].apply(lambda x: x)
d.label = True
d.loc[d['Epicolon1+2IHQ-IMS-maria paper lynlike_IMS'] == "inestable",'label' ] = False

In [ ]:
d.label.describe()

In [ ]:
d.shape[0]

In [ ]:
j = 567
print((d.iloc[j][['tile','path', 'tif_fn']]))
p = Spot.get_tile(Spot, *(d.iloc[j].tile), path = d.iloc[j].path, tif_name = d.iloc[j].tif_fn)
print(np.array(p).shape)
p

In [ ]:
import torch
print(np.max(np.array(p)[:,:,3]))
p1 = np.array(p)[:,:,:3]


img = np.array(p)[:,:,:3]
t = torch.Tensor(np.array(img))
t= t.permute(2,0,1).float()/255.0
t.shape,t

In [ ]:
idxs = np.random.permutation(range(d.shape[0]))
cut = int(0.8 * d.shape[0])
train_files = d.iloc[idxs[:cut]]
valid_files = d.iloc[idxs[cut:]]
valid_files

In [ ]:
class TileDataset(torch.utils.data.Dataset):
    def __init__(self, df, is_valid=False):
        self.tiles,self.is_valid = df,is_valid
        
    def __getitem__(self, i):
        self.tiles
        
        img = Spot.get_tile(Spot, *(self.tiles.iloc[i].tile), path = self.tiles.iloc[i].path, tif_name = self.tiles.iloc[i].tif_fn)
        #print(img)
        if (isinstance(img, Image.Image) == False): 
            i = j
            img = Spot.get_tile(Spot, *(self.tiles.iloc[i].tile), path = self.tiles.iloc[i].path, tif_name = self.tiles.iloc[i].tif_fn)
        
        img = np.array(img)[:,:,:3]
        t = torch.Tensor(np.array(img))
        t = t.permute(2,0,1).float()/255.0
        
        label = torch.Tensor([self.tiles.iloc[i].label]).squeeze()
        return (TensorImage(t), label)
    
    def __len__(self): return self.tiles.shape[0]
        
        

In [ ]:
train_ds = TileDataset(train_files)
valid_ds = TileDataset(valid_files, is_valid=True)

In [ ]:
from fastai2.data.core import DataLoaders

In [ ]:
dls = DataLoaders.from_dsets(train_ds, valid_ds,device=default_device(),
                            after_batch=[Normalize.from_stats(*imagenet_stats), ])

In [ ]:
b = dls.one_batch()

In [ ]:
b[0]

In [ ]:
explode_types(b)

In [ ]:
#learn = cnn_learner(dls, resnet34,config={'ps':0.}, loss_func=MSELossFlat(),metrics=mse,n_out=1)
learn = cnn_learner(dls, resnet34,config={'ps':0.}, loss_func=BCEWithLogitsLossFlat() ,metrics=error_rate ,n_out=1)

In [ ]:
learn.summary()

In [ ]:
learn.lr_find()

In [ ]:
learn.recorder.plot_lr_find()

In [ ]:
learn.fit_one_cycle(10,1e-2)

In [ ]:
dls[1].dataset[0]

In [ ]:
dls[1].dataset[0]

In [ ]:
learn.get_preds()[1].shape

In [ ]:
interp = Interpretation.from_learner(learn)

In [ ]:
interp.__dict__


In [ ]:
interp.losses.shape

In [ ]:
tma = tmas[1]

print(tma.meta_path)
print(tma.img_path_tif)

print(tma.spots[2].center)
spot = tma.spots[5]
jpg = spot.load_img_jpg()
tif = tma.load_img_tif()
tif

In [ ]:
tma.img_tif.properties
print(spot.TMA.name)
print(spot.name)
print(spot.ID)
print(spot.diameter)
spot.get_center_pixel()
print(spot.get_radious_pixel(0))
print(spot.get_circle_coordinates_pixel(0))
print(spot.check_tile_in_spot(41100, spot.get_circle_coordinates_pixel(0)[1] - spot.get_radious_pixel(0)[0]))
spot.get_tile(41100, spot.get_circle_coordinates_pixel(0)[1] - spot.get_radious_pixel(0)[0])

In [ ]:
img = spot.get_enclosing_box(4)
print(img.size)
img

In [ ]:
spot.tiles = []
print(len(spot.tiles))
for t in spot.sample_tiles(size = (300,300), level = 2, overlap = .3):
    print(t)
    

In [ ]:
print(len(spot.tiles))
print(spot.tiles[1])

In [ ]:
plt.imshow(spot.get_tile(*spot.tiles[0]))

In [ ]:
spot.get_tile(*spot.tiles[0])

In [ ]:
tma.img_tif.properties